In [2]:
install.packages('foqat')

Installing package into ‘/home/millieginty/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)

also installing the dependencies ‘gridGraphics’, ‘yulab.utils’, ‘plyr’, ‘lmodel2’, ‘reshape2’, ‘ggplotify’, ‘ggnewscale’, ‘patchwork’




In [8]:
# Written by Matthew CH Vaughan
# edited/adapted by Erin Seybold on 3/16/18
# edited by SB, SS on 02292024
# editied by Megan Duffy on 05/28/24

library(ggplot2)
library("tidyverse")
library("lubridate")
library("foqat")


# -------------------------------------------------------------------------  
# -------------------------------------------------------------------------  
# ------------------------------------- WADE ------------------------------  
# User inputs -------------------------------------------------------------

#setwd("/Users/mat/OneDrive - University of Vermont/Watershed Data/Streams/02_site_data/discharge") # for MAC
# setwd("/Users/dustinkincaid/OneDrive - University of Vermont/Watershed Data/Streams/02_site_data/discharge")
# Import data

# Import data
setwd("/home/millieginty/Documents/git-repos/cQ_analysis/cond-hydrograph-separation/newrnet/data/")

#### - edited by SB and SS 02292024

source("AquariusR/aquariusHelperFunctions.R", chdir = T)

aquariusLoginWithFile("AquariusR/aquariusLogin")


df1 <- getCorrectedAquariusDataSeries(
  "Sp Cond.SpecCond@Wade Brook",
  startTime = "2022-01-01T00:00:00-05:00",
  endTime   = "2022-12-31T00:00:00-05:00")

df1$timestamp <-df1$Timestamp

 # Export q data
write.csv(df1, "hungerford_2022_sC.csv")

In [4]:
df2 <- getCorrectedAquariusDataSeries(
  "Depth.Staff Ruler Height@Wade Brook",
  startTime = "2020-01-01T00:00:00-05:00",
  endTime   = "2023-12-31T00:00:00-05:00")

df2$timestamp <-df2$Timestamp

#### - edited end ---- SB and SS 02292024

# 
# manual_staff_data <- read_csv("Wade_StaffGauge_2017-2022.csv", col_types = cols())  %>% 
#   mutate(r_timestamp =  mdy_hm(timestamp, tz = "Etc/GMT+4"))



manual_staff_data <- df2 # edited by SB SS


stage_discharge_filename <- "scripts/20160908_wade_stage_q_data.csv" #rating curve info 
# hobo_stage_data_filename <- "Wade_Stage_2022_COMPILED.csv" #raw stage data - change this to file for analysis
#staff_gauge_manual_readings <- "Staff_gauge_readings/Wade_StaffGauge_2017-2020.csv"

hobo_stage_data_filename <- df1 # edited by SB SS

dist_from_hobo_to_streambed <- 0.221716 # In meters. See excel spreadsheet for data+details on this calculation

# Load data ---------------------------------------------------------------
  # curve_data <- read.csv(stage_discharge_filename) - commented by SB SS Feb 2024
  # sensor_data <- read.csv(hobo_stage_data_filename) - commented by SB SS Feb 2024
#  manual_staff_data <- read.csv(staff_gauge_manual_readings)
  
# edited by SB SS 02292024
  curve_data <- read.csv(stage_discharge_filename)
  sensor_data <-df1 
# end edit by SB SS 02292024

 # Adjust hobo stage to reflect distance above the streambed
  curve_data$stage <- curve_data$hobo_stage + dist_from_hobo_to_streambed
  
  # for 2014-2017 - sensor_data doesn't needisd any additional processing other than the distance offset, for 2018 and 2019 see below.
  
  # for 2018
  #sensor_data$hobo_stage_int <- 1.1225*sensor_data$hobo_stage + 0.164 # specific to 2018 - additional offset to correct meas HOBO stage to manual measurements
  # see file "WadeStage_2018_COMPILED_manual stage comp" for details on this correction (located in rating curve folder)
  
  # for 2019
  #sensor_data$hobo_stage_int <- 0.9967*sensor_data$hobo_stage + 0.063 # specific to 2019 - additional offset to correct meas HOBO stage to manual measurements
  # see file "WadeStage_2019_COMPILED_manual stage comp" for details on this correction (located in rating curve folder)
 
  # for 2020 data stage was corrected in aquarius using staff gauge readings
  # sensor_data$Stage <- sensor_data$stage + dist_from_hobo_to_streambed
  
  # for 2020 
  #sensor_data$hobo_stage_int <- 0.9163*sensor_data$hobo_stage + 0.1026 # specific to 2020 - additional offset to correct meas HOBO stage to manual measurements
  # see file "Wade_stage_manual_offset_2020" for details on this correction (located in rating curve folder)

  # for 2021 REVISED
  #sensor_data$hobo_stage_int <- 0.876*sensor_data$hobo_stage + 0.1077 # specific to 2021 - additional offset to correct meas HOBO stage to manual measurements
  # see file "Wade_stage_manual_offset_2021" for details on this correction (located in rating curve folder)

  # for 2022 REVISED
  #corrections already applied to "Wade_Stage_2022_COMPILED.csv" file, see file "Wade_stage_manual_offset_2021" for details on this correction (located in rating curve folder)
    
  # sensor_data$stage <- sensor_data$hobo_stage + dist_from_hobo_to_streambed # for 2014-2018 years, this is the final stage and can be renamed/line below can be commented out
  
  sensor_data$stage <- sensor_data$'Stage.Stage@Wade Brook' + dist_from_hobo_to_streambed # for 2014-2018 years, this is the final stage and can be renamed/line below can be commented out
  
  
  
  # Convert timestamps
 # sensor_data$r_timestamp <- as.POSIXct(strptime(sensor_data$timestamp,'%m/%d/%y %H:%M', tz = "Etc/GMT-4"))
 # sensor_data$timestamp <- NULL
 
  sensor_data$r_timestamp <- sensor_data$timestamp
 curve_data$r_timestamp <- as.POSIXct(strptime(curve_data$timestamp,'%m/%d/%Y %H:%M', tz = "Etc/GMT-4"))
 curve_data$timestamp <- NULL

# manual_staff_data$r_timestamp <- as.POSIXct(strptime(manual_staff_data$timestamp,'%m/%d/%y %H:%M', tz = "Etc/GMT-4"))
# # manual_staff_data$timestamp <- NULL
 
 manual_staff_data$r_timestamp <- manual_staff_data$timestamp
 manual_staff_data$stage <- manual_staff_data$`Depth.Staff Ruler Height@Wade Brook`
 
#manual_staff_data$stage <- manual_staff_data$Staff.level..m.
#manual_staff_data$Staff.level..m. <- NULL

# Tidy up manual_staff_data for comparison
manual_staff_data <-
  manual_staff_data%>% 
  mutate(r_timestamp = round_date(r_timestamp, unit = "15 mins")) %>% 
  select(timestamp, stage) %>% 
  mutate(r_timestamp = ymd_hms(timestamp))

 
# Compare sensor-derived staget to manual stage readings ----------------------------
 # Join data
 allstage <- 
   sensor_data %>% 
   select(r_timestamp, stage) %>% 
   rename(stage_sensor = stage) %>% 
   full_join(manual_staff_data %>% 
               select(r_timestamp, stage) %>% 
               rename(stage_manual = stage) %>% 
               mutate(r_timestamp = round_date(r_timestamp, unit = "15 mins")))
 
 # Plot
 allstage %>% 
   ggplot(aes(x = stage_manual, y = stage_sensor)) +
   geom_point() +
   geom_abline(slope = 1)+
   ylim(0,1.5)+
   xlim(0,1.5)
 
 
# Calculate the interpolated rating curve --------------------------------------------
 # First make interpolated rating curve within measurement bounds
  max_measured_stage <- max(curve_data$stage)

 # Perform the power function fit
  fit <- nls(data = curve_data, q ~ a * stage^b,  start = list(a=1, b=1))
  fit_coeff <- summary(fit)$coefficients
  
  # Pull out the coefficients
  a <- fit_coeff[1,1]
  b <- fit_coeff[2,1]
  
 # Create function for curve
  power_curve <- function(x) {a * x^b}
  
  sensor_data$power_q <- power_curve(sensor_data$stage)
    
 # Find the predicted q based on measured stages and the rating curve. 
  curve_data$predicted_q <- predict(fit)

 # Plot to check
  p <- ggplot(data = curve_data)
  p <- p+ geom_point(aes(x = stage, y = q, shape = type, color = r_timestamp), size = 3)
  p <- p+ stat_function(fun = power_curve, color = "black", size = 1.5)
  p

# Calculate the extrapolated rating curve ---------------------------------
  # See survey notes for where these values came from. took averages where appropriate
  inner_width <- 7.05
  top_lip <- 0.2275
  culvert_diam <- inner_width + top_lip
  culvert_radius <- culvert_diam / 2
  berm_height <- 1.234 # from bottom to where the currogated circular culvert begins.
  
  # Function to find the wetted area of the culvert above the berm
    A_wet <- function(stage, culvert_radius = 3.63875, berm_height = 1.234, inner_width = 7.05) {
      # Create empty result vector to populate
      result <- numeric(length(stage))
      # Start for loop to go through each element of stage.
      for (i in 1:length(stage)) {
      # See if stage is above berm height
      if ((stage[i] - berm_height) > 0) {
        # If it is, then find the distance above berm height (y)
           y <- stage[i] - berm_height
           # And calculate the wetted area below berm heigh (simple rectangle)
           A_below_berm <- inner_width * berm_height
      # If it's not...
      } else { 
        # Set the distance above the berm to zero
             y <- 0
        # And calculate the are below based on stage (rectangular area depends on how high the stage is)
             A_below_berm <- inner_width * stage[i]
      } # End else statement
      
     # Now calculate area above berm if there is any (will be 0 if stage is below berm)
      A_sector <- culvert_radius^2 * (pi - 2*asin(y / culvert_radius)) / 2
      A_triangle <- y * sqrt(culvert_radius^2 - y^2)
      semi_circ <- (1/2) * pi * culvert_radius^2
      A_above_berm <- semi_circ - (A_sector - A_triangle)
      
      # Add them together for the result.
      result[i] <- A_below_berm + A_above_berm
      } # End for loop     
      
      # Return the result
      return(result)
    } # End function
    
  # Function to find hydraulic radius (wetted perimeter)
    R_wet <- function(stage, culvert_raius = 3.63875, berm_height = 1.234, inner_width = 7.05, top_lip = 0.2275) {
      # Create empty result vector to populate
      result <- numeric(length(stage))
      # Start for loop to go through each element of stage.
      for (i in 1:length(stage)) {
      # See if stage is above berm height
      if ((stage[i] - berm_height) > 0) {
        # If it is, then find the distance above berm height (y)
        y <- stage[i] - berm_height
        # And calculate the wetted perimeter below berm heigh (simple rectangle)
        R_below_berm <- inner_width + (2*berm_height)
        # If it's not...
      } else { 
        # Set y to zero
        y <- 0
        # Calculate wetted perimeter based on stage
        R_below_berm <- inner_width + (2*stage[i])
        # Set the top_lip dimension to zero
        top_lip <- 0
      }
      
      # Now calculate wetted perimeter above berm if there is any
      R_above_berm <- (2*top_lip) + culvert_radius * asin(y / culvert_radius)
      # Add them together for result
      result[i] <- R_below_berm + R_above_berm
      } # End for loop
      
      return(result)
    } # End function
    
    # Calculate the wetted areas and radii of the sensor and curve data measurements
  curve_data$radius <- R_wet(curve_data$stage)
  sensor_data$radius <- R_wet(sensor_data$stage)
  
  curve_data$area <- A_wet(curve_data$stage)
  sensor_data$area <- A_wet(sensor_data$stage)
    
  # Determeine whether the q measurement is interpolated or extrapolated based on max stage/discharge pair
  sensor_data[which(sensor_data$stage <= max_measured_stage), "int_or_ext"] <- "interpolated"
  sensor_data[which(sensor_data$stage > max_measured_stage), "int_or_ext"] <- "extrapolated"
  
  # Slope across reach calculated by Ryan
  slope <- 0.0244
  
  curve_data$back_calc_n <- curve_data$area * curve_data$radius^(2/3) * sqrt(slope) / curve_data$q
  
  n_rough <- 1.355
  
  sensor_data$mannings_q <- (1 / n_rough) * sensor_data$area * sensor_data$radius ^ (2/3) * sqrt(slope)
  
# Combine two methods to make best curve ----------------------------------

 break_point <- max_measured_stage
  
 int <- sensor_data$stage <= break_point
 ext <- sensor_data$stage > break_point
 
  max(sensor_data$power_q[int])
  min(sensor_data$mannings_q[ext])
 
 sensor_data$best_q <- NA
 sensor_data$best_q[int] <- sensor_data$power_q[int]
 # Bring manning's curve down a hair so that the transition is more continuous.
 sensor_data$best_q[ext] <- sensor_data$mannings_q[ext] - 0.8*(min(sensor_data$mannings_q[ext]) - max(sensor_data$power_q[int]))

  e <- ggplot(data = subset(curve_data, stage > 0.6))
  e <- e+ geom_density(aes(x = back_calc_n))
  e
  
  s <- ggplot(data = sensor_data)
  s <- s+ geom_point(aes(x = r_timestamp, y = best_q, color = int_or_ext), size = 2)
  s
  
  s <- ggplot(data = sensor_data)
  s <- s+ geom_point(aes(x = stage, y = best_q, color = int_or_ext), size = 2)
  s
  
  # Export q data
  write.csv(sensor_data, "wade_2023_best_q_corr_SBSS.csv")

  # -------------------------------------------------------------------------  
  # -------------------------------------------------------------------------  
  # ------------------------------- POTASH ----------------------------------  
  # User inputs -------------------------------------------------------------
  
  # #setwd("/Users/erinseybold/Dropbox/R/BREE_R/NN_rating_curves") # for MAC 
  # #2020 use wd from above-sb
  # PB_stage_data_filename <- "2020_Potash_Stage_corrected_sb.csv" #raw stage data - change this to file for analysis
  # 
  # # dist_from_hobo_to_streambed --> NO depth offset for Potash
  # 
  # # Load data ---------------------------------------------------------------
  # PB_sensor_data <- read.csv(PB_stage_data_filename)
  # 
  # # Convert timestamps
  # PB_sensor_data$r_timestamp <- as.POSIXct(strptime(PB_sensor_data$timestamp,'%m/%d/%y %H:%M', tz = "Etc/GMT-4"))
  # PB_sensor_data$timestamp <- NULL
  # 
  # PB_sensor_data$PB_best_q <- 3.0386*(PB_sensor_data$stage^6.5671)
  # 
  # # Export q data
  # write.csv(PB_sensor_data, "potash_2020_best_q.csv")
  
  # -------------------------------------------------------------------------  
  # -------------------------------------------------------------------------  
  # ----------------------------- HUNGERFORD --------------------------------  
  # User inputs -------------------------------------------------------------
  
  # edited by SB and SS Feb-Mar 2024
  
  
  # Import data
  setwd("C:/Users/sshercha/OneDrive - University of Vermont/Desktop/discharge_new")
  
  #### - edited by SB and SS 02292024
  
  source("AquariusR/aquariusHelperFunctions.R", chdir = T)
  
  aquariusLoginWithFile("AquariusR/aquariusLogin")
  
  
  df1 <- getCorrectedAquariusDataSeries(
    "Stage.StagemHOBO@Hungerford Brook",
    startTime = "2020-01-01T00:00:00-05:00",
    endTime   = "2023-12-31T00:00:00-05:00")
  
  df1$timestamp <-df1$Timestamp
  
  df2 <- getCorrectedAquariusDataSeries(
    "Stage.RelDepth@Hungerford Brook",
    startTime = "2020-01-01T00:00:00-05:00",
    endTime   = "2023-12-31T00:00:00-05:00")
  
  df2$timestamp <-df2$Timestamp
  
  #### - edited end ---- SB and SS 02292024
  
  # 
  # manual_staff_data <- read_csv("Wade_StaffGauge_2017-2022.csv", col_types = cols())  %>% 
  #   mutate(r_timestamp =  mdy_hm(timestamp, tz = "Etc/GMT+4"))
  
  
  
  manual_staff_data <- df2 # edited by SB SS
  
  hobo_stage_data_filename <- df1 # edited by SB SS
  
  
  # #setwd("/Users/erinseybold/ownCloud/BREE (2)/Watershed Data/Streams/02_site_data/discharge") # for MAC
  # 
  # # 2020 use wd from above-sb
  # HF_stage_data_filename <- "HB_Stage_2022_COMPILED.csv" #raw stage data - change this to file for analysis
  # 
  
  # Load data ---------------------------------------------------------------
  HF_sensor_data <- df1
  
  # Convert timestamps
  # HF_sensor_data$r_timestamp <- as.POSIXct(strptime(HF_sensor_data$timestamp,'%m/%d/%y %H:%M', tz = "Etc/GMT-4"))
  # HF_sensor_data$timestamp <- NULL
  HF_sensor_data$r_timestamp <- HF_sensor_data$Timestamp
  
  
  
  HF_sensor_data$offset <- 0.1597 # this corrects the measured stage to the USGS stage so that the USGS rating curve can be used on NEWRnet stage time series
  
  #HF_sensor_data$corr_stage <- HF_sensor_data$hobo_stage-HF_sensor_data$offset # only need this for years 2014-20189
  
  #HF_sensor_data$corr_stage_2019 <- 0.793*HF_sensor_data$corr_stage + 0.2366 # specific to 2019 - additional offset to correct meas HOBO stage to manual measurements
  # see file "HfordStage_2019_COMPILED_manual stage comp" for details on this correction (located in rating curve folder)
  ### 2020 stage data corrected in aquarius using staff gauge readings no offset or correction needed in script-sb
  
  HF_sensor_data$stage <- HF_sensor_data$'Stage.StagemHOBO@Hungerford Brook' - 0.1597
  
  HF_sensor_data$HF_best_q <- 9.736*(HF_sensor_data$stage^2.2984)
  
  ggplot(HF_sensor_data, aes(r_timestamp,HF_best_q))+
    geom_point(aes(), alpha = 0.8,  size = 2, shape = 21, colour = "black")+
    ylim(0,4)
  
  # Export q data
  write.csv(HF_sensor_data, "hungerford_2023_best_q.csv")

Loading required package: jsonlite


Attaching package: ‘jsonlite’


The following object is masked from ‘package:purrr’:

    flatten


Loading required package: httr

Warning message in file(file, "rt"):
“cannot open file 'scripts/20160908_wade_stage_q_data.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection
